In [1]:
import torch
import numpy as np
from tqdm import tqdm
from loguru import logger

# check if gpu available
logger.info(torch.cuda.is_available())

device = torch.has_cuda
# print gpu name
if device:
    logger.info(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    logger.info('Using CPU')

2023-09-21 06:20:11.747 | INFO     | __main__:<module>:7 - True
2023-09-21 06:20:11.750 | INFO     | __main__:<module>:12 - GPU Name: Quadro M4000


### DATA

In [5]:
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

mnist_transform = transforms.Compose([transforms.ToTensor()])

dataset_path = "../datasets/"
train_set = MNIST(dataset_path, train=True, transform=mnist_transform, download=True)
val_set = MNIST(dataset_path, train=False, transform=mnist_transform, download=True)

train_loader = DataLoader(train_set, 16, shuffle=True, pin_memory=True, num_workers=1)
val_loader = DataLoader(val_set, 16, shuffle=True, pin_memory=True, num_workers=1)

for batch_idx, (x,y) in enumerate(train_loader):
    logger.info(f"X shape: {x.shape}")
    logger.info(f"Y shape: {y.shape}")
    break


2023-09-21 06:21:33.031 | INFO     | __main__:<module>:15 - X shape: torch.Size([16, 1, 28, 28])
2023-09-21 06:21:33.035 | INFO     | __main__:<module>:16 - Y shape: torch.Size([16])


### MODEL

In [6]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.x1 = nn.Linear(input_dim, 1024)
        self.x2 = nn.Linear(1024, 512)


IndentationError: expected an indented block (3720170636.py, line 4)